In [15]:
import sys
import os
parent_dir = os.path.abspath('..')
sys.path.append(parent_dir)
import system.robot as robotSystem
import casadi as ca
import system.utils.transformation_matrix as Transformation
import numpy as np

In [16]:
alpha = robotSystem.RobotDynamics()
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
path_to_urdf = os.path.join(
    project_root,
    'usage',
    'urdf','reach_alpha_5',
    'alpha_5_robot.urdf'
)
alpha.from_file(path_to_urdf)
root = "base_link"
tip = "alpha_standard_jaws_base_link"
kinematic_dict, K, P, L, D, C, g, B, qdd, joint_torque, sys_id_coeff, F_payload = alpha.build_model(root, tip, floating_base=True)
kinematic_dict = alpha.get_kinematic_dict()
c_parms, m_params, I_params, fv_coeff, fc_coeff, fs_coeff, v_s_coeff, vec_g, r_com_body, m_p, q, q_dot, q_dotdot, tau, base_pose, world_pose, tip_offset_pose = kinematic_dict['parameters']
n_joints = alpha.get_n_joints(root, tip)

In [17]:
# decision variables for IK, these are spatial velocities, not Euler rates
twist_base_W = ca.SX.sym('twist_base_W', 6, 1)  
v_base_W     = twist_base_W[0:3] # base linear velocity in world
omega_base_W = twist_base_W[3:6] # base angular velocity in world

q_dot_cmd = ca.SX.sym('q_dot_cmd', n_joints, 1)
nu = ca.vertcat(twist_base_W, q_dot_cmd)

# position task
Jp_ee_W = kinematic_dict['geo_J_full'][-1][0:3, :]    # linear Jacobian (world)
p_ee_W  = kinematic_dict['Fks'][-1][0:3]              # EE position in world

kp = ca.SX.sym('kp', 3, 1)
p_des = ca.SX.sym('p_des', 3, 1)

# desired end-effector linear velocity (per-axis gains)
v_des = ca.diag(kp) @ (p_des - p_ee_W)                # 3 x 1

# task residual: match task-space linear velocity
e_p_task = Jp_ee_W @ nu - v_des                       # 3 x 1

In [ ]:
# build a full desired rotation and use a rotation error
# Desired tool axes expressed in world frame
a_des_x = ca.SX.sym('a_des_x', 3, 1)
a_des_y = ca.SX.sym('a_des_y', 3, 1)
a_des_z = ca.SX.sym('a_des_z', 3, 1)

# Normalize desired axes
x_d = a_des_x / (ca.norm_2(a_des_x) + 1e-12)
y_d = a_des_y / (ca.norm_2(a_des_y) + 1e-12)
z_d = a_des_z / (ca.norm_2(a_des_z) + 1e-12)

# Orthonormalize, prioritize z, then x, then y (Gram Schmidt)
x_d = x_d - z_d * (z_d.T @ x_d)
x_d = x_d / (ca.norm_2(x_d) + 1e-12)

# Build a right handed orthonormal frame
y_d = ca.cross(z_d, x_d)
y_d = y_d / (ca.norm_2(y_d) + 1e-12)
x_d = ca.cross(y_d, z_d)

# Desired tool orientation in world, columns are tool axes
R_des = ca.horzcat(x_d, y_d, z_d)

# Current tool orientation in world
R_ee_W = kinematic_dict['R_symx'][-1]

# Spatial orientation error in world frame
E = R_ee_W @ R_des.T - R_des @ R_ee_W.T
e_R_W = 0.5 * ca.vertcat(E[2,1], E[0,2], E[1,0])

# Convert orientation error into desired angular velocity
k_axis = ca.SX.sym('k_axis')
w_des_axis = -k_axis * e_R_W

# Angular Jacobian mapping nu to tool angular velocity in world
Jw_ee_W = kinematic_dict['geo_J_full'][-1][3:6, :]

# Residual enforcing the orientation task
e_axis_task = Jw_ee_W @ nu - w_des_axis

In [19]:
# roll,pitch soft stabilization (servo Euler angles back toward 0)
phi   = world_pose[3]
theta = world_pose[4]
psi = world_pose[5]
k_rp = ca.SX.sym('k_rp')                         # scalar

# desired Euler rates (stabilize roll, pitch, leave yaw free)
phi_dot_des   = k_rp * (0 - phi)
theta_dot_des = k_rp * (0 - theta)
psi_dot_des   = 0

# your tested mapping: omega_world = Ts(phi,theta,psi) @ euler_dot
Ts = Transformation.euler_to_spatial_rate_T(phi, theta, psi)
euler_dot_des = ca.vertcat(phi_dot_des, theta_dot_des, psi_dot_des)
omega_des_rp  = Ts @ euler_dot_des                        # 3x1, desired world angular velocity

# residual in angular velocity space (consistent with geo Jw)
e_omega = omega_base_W - omega_des_rp   # 3x1

# only penalize roll and pitch behavior, do not fight yaw
e_rp    = e_omega[0:2]                  # 2x1

In [20]:
# relative orientation task between the floating base and the arm’s first link to keep it centered
a_x = ca.DM([1, 0, 0])                      # local x axis

R_B_W  = Transformation.rot_from_rpy(phi, theta, psi)     # base rotation in world
R_J0_W = kinematic_dict['R_symx'][0]                      # joint0 rotation in world

x_B_W  = R_B_W  @ a_x                      # base x axis in world
x_J0_W = R_J0_W @ a_x                     # joint0 x axis in world

Jw_J0_W    = kinematic_dict['geo_J_full'][0][3:6, :]  # joint0 angular Jacobian
omega_J0_W = Jw_J0_W @ nu                 # joint0 angular velocity in world

omega_rel = omega_J0_W - omega_base_W    # joint0 relative to base

e_align = ca.cross(x_J0_W, -x_B_W)        # axis misalignment error
k_align = ca.SX.sym('k_align')
w_des_align = k_align * e_align           # desired relative angular velocity

e_align_task = omega_rel - w_des_align    # alignment residual

In [21]:
# regularization (keeps solution small, avoids base thrashing)
e_reg = nu                                       # (6+n) x 1

In [22]:
# weighted least squares objective
w_rp  = ca.SX.sym('w_rp')                        # scalar
w_reg = ca.SX.sym('w_reg')                       # scalar
w_axis = ca.SX.sym('w_axis')  # scalar
w_align = ca.SX.sym('w_align')
cost = (
    ca.dot(e_p_task, e_p_task)
    + w_axis * ca.dot(e_axis_task, e_axis_task)
    + w_rp  * ca.dot(e_rp, e_rp)
    + w_reg * ca.dot(e_reg, e_reg)
    + w_align * ca.dot(e_align_task, e_align_task)
    
)

In [23]:
# 1. Compute Hessian and Full Gradient
H, g_full = ca.hessian(cost, nu)

# 2. Extract the linear term of the gradient
# The full gradient is (H * nu + g_linear). 
# By substituting nu=0, we isolate g_linear.
g_linear = ca.substitute(g_full, nu, ca.DM.zeros(nu.shape))

# 3. Solve for the optimal nu
# Analytic solution for QP: H * nu + g_linear = 0  =>  nu = -inv(H) * g_linear
lam = 1e-8 + 1e-6 * w_reg
nu_star = -ca.solve(H + lam * ca.SX.eye(H.size1()), g_linear)

In [24]:
dt = ca.SX.sym('dt')

# linear part integrates directly
p_world_next = world_pose[0:3] + nu_star[0:3] * dt

# convert omega -> euler_dot for integration
Ts = Transformation.euler_to_spatial_rate_T(phi, theta, psi)

euler_dot_star = ca.solve(Ts, nu_star[3:6])               # 3x1

rpy_next = world_pose[3:6] + euler_dot_star * dt



x_world_ref_next = ca.vertcat(p_world_next, rpy_next)
q_ref_next       = q + nu_star[6:6+n_joints] * dt

# Pack state = [world_pose; q]
state0 = ca.vertcat(world_pose, q)  # (6+n) x 1

# One IK step as a CasADi function (state -> next_state)
state1 = ca.vertcat(x_world_ref_next, q_ref_next)  # (6+n) x 1

e_p_task_star = Jp_ee_W @ nu_star - v_des
e_axis_task_star = Jw_ee_W @ nu_star - w_des_axis

ik_step = ca.Function(
    'ik_step',
    [state0, kp, p_des, w_rp, w_reg, k_rp, a_des_x, a_des_z, k_axis, w_axis, w_align, k_align, dt, base_pose, tip_offset_pose],
    [state1, e_p_task_star, e_axis_task_star]
)

In [25]:
# Mapaccum over N iterations
N = 5000
ik_step_acc = ik_step.mapaccum('ik_step_acc', N)

# Accumulated trajectories
state_traj, e_p_traj, e_axis_traj = ik_step_acc(
    state0, kp, p_des, w_rp, w_reg, k_rp, a_des_x, a_des_z, k_axis, w_axis, w_align, k_align, dt, base_pose, tip_offset_pose
)

In [26]:
# Take the final values (last column)
stateN = state_traj[:, -1]
x_world_next = stateN[0:6]
q_next       = stateN[6:6+n_joints]

e_p_task_star_new    = e_p_traj[:, -1]
e_axis_task_star_new = e_axis_traj[:, -1]

ik_func = ca.Function(
    'mapacc_task_based_ik',
    [q, world_pose, kp, p_des, w_rp, w_reg, k_rp,  a_des_x, a_des_z, k_axis, w_axis, w_align, k_align, dt, base_pose, tip_offset_pose],
    [x_world_next, q_next, e_p_task_star_new, e_axis_task_star_new]
)

In [27]:
q_val=[1.0002, 1.9335, 0.3438, 0.0125]
world_pose_val=[ 4.1448e-01 ,-1.5551e-01, -1.3213e+00,  3.7661e-05,  3.1087e-04 , 9.7403e-01]
p_des_val=[ 5.5038e-01 ,-1.2290e-17 ,-1.5958e+00]
a_des_z_val = ca.DM([0, 0, -1])   # tool z axis points downward
a_des_x_val = ca.DM([1, 0, 0])    # tool x axis aligned with world x


kp_val = np.array([1.0, 1.0, 1.0], dtype=float)
w_rp_val = 1.0
w_reg_val = 0.02
k_rp_val = 0.2
k_axis_val = 1.0
w_axis_val = 1.5
dt_val = 1.0 / 500.0
w_align_val = 1.0
k_align_val = 1.0

base_pose_val = np.asarray([0.190, 0.000, -0.120, 3.141592653589793, 0.000, 0.000], dtype=float)
tip_offset_pose_val = np.asarray([0.00, 0.00, 0.04, 0.00, 0.00, 0.00], dtype=float)

ik_func(q_val, world_pose_val, kp_val, p_des_val, w_rp_val, w_reg_val, k_rp_val,
         a_des_x_val, a_des_z_val,
         k_axis_val, w_axis_val, w_align_val, k_align_val,
           dt_val, base_pose_val, tip_offset_pose_val)

(DM([0.826073, -0.26097, -1.28149, -5.1894e-05, -0.00546306, 2.38356]),
 DM([3.14137, 1.1457, 1.14012, -0.289346]),
 DM([2.56798e-07, -1.93683e-07, -1.14021e-06]),
 DM([2.4086e-05, -2.59076e-05, -1.12669e-06]))

In [28]:
ik_func.save('ik_whole_b.casadi')